In [ ]:
!pip install import-ipynb

**STEP1: Load google-drive, and libraries**

In [ ]:
# Mounting google drive if it is already not mounted
def LoadGoogleDrive(googleDriveDir):
  # Link Google Drive
  import os
  # Check if Google Drive is already mounted
  if not os.path.exists(googleDriveDir + "/MyDrive"):
      print("Mounting Google Drive...")
      from google.colab import drive
      drive.mount(googleDriveDir)
  else:
      print("Google Drive is already mounted!")

from google.colab import drive
googleDriveFolder = "/content/drive"
LoadGoogleDrive(googleDriveFolder)

Mounting Google Drive...
Mounted at /content/drive


In [ ]:
import sys
import import_ipynb

# 1. Set path to the directory where mylib.ipynb is located
lib_path = '/content/drive/MyDrive/Colab Notebooks/P_HINTS6/lib'
sys.path.append(lib_path)
# 2. Change current working directory to that path
%cd "$lib_path"

# 3. Now import the notebook
from app_lib import *


/content/drive/MyDrive/Colab Notebooks/P_HINTS6/lib


**STEP2: Load data**

In [ ]:
ip_file_path = googleDriveFolder + "/MyDrive/Colab Notebooks/P_HINTS6/ip_data/hints6_public_filtered_v1.xlsx"
# ip_sheet_name = 'hints6_public_colAllRowFilter'
ip_sheet_name = 'hints6_public_colFilter'

df_orig = load_data(verbose_level=1, ip_file_path=ip_file_path, ip_sheet_name=ip_sheet_name)

Ip File Path: /content/drive/MyDrive/Colab Notebooks/P_HINTS6/ip_data/hints6_public_filtered_v1.xlsx
 SheetName: hints6_public_colFilter
df_orig.shape: (6252, 19)


In [ ]:
df = df_orig.copy()

In [ ]:
# Step 2: Drop rows with any negative values in numeric columns
df = df[~(df.select_dtypes(include=[np.number]) < 0).any(axis=1)]


In [ ]:
df.shape

(4865, 19)

**All Columns:**

FreqGoProvider
GeneralHealth
Deaf
MedConditions_Diabetes
MedConditions_HighBP
MedConditions_HeartCondition
MedConditions_LungDisease
MedConditions_Depression
AverageSleepNight
AverageTimeSitting
EverHadCancer
Age
BirthGender
BMI
smokeStat
PHQ4
WeeklyMinutesModerateExercise
eCigUse
AvgDrinksPerWeek


In [ ]:
# ColumName: 'GeneralHealth'

# Define mapping of numeric values to labels
health_labels = {
    1: 'Excellent',
    2: 'VeryGood',
    3: 'Good',
    4: 'Fair',
    5: 'Poor'
}

# Create one-hot encoded columns with custom names
for code, label in health_labels.items():
    col_name = f"GeneralHealth_{label}"
    df[col_name] = (df['GeneralHealth'] == code).astype(int)

# Drop the original column
df.drop(columns=['GeneralHealth'], inplace=True)

In [ ]:
# List of medical condition columns
med_columns = [
    'Deaf',
    'MedConditions_Diabetes',
    'MedConditions_HighBP',
    'MedConditions_HeartCondition',
    'MedConditions_LungDisease',
    'MedConditions_Depression',
    'EverHadCancer',
    'BirthGender'
]

# Replace 2 with 0 in all specified columns
df[med_columns] = df[med_columns].replace(2, 0)


In [ ]:
# Column Names: 'smokeStat', 'eCigUse'

# Define mapping of numeric values to labels
health_labels = {
    1: 'Current',
    2: 'Former',
    3: 'Never'
}

# Create one-hot encoded columns with custom names
for code, label in health_labels.items():
    col_name = f"smokeStat_{label}"
    df[col_name] = (df['smokeStat'] == code).astype(int)

# Create one-hot encoded columns with custom names
for code, label in health_labels.items():
    col_name = f"eCigUse_{label}"
    df[col_name] = (df['eCigUse'] == code).astype(int)

# Drop the original column
df.drop(columns=['smokeStat', 'eCigUse'], inplace=True)

In [ ]:
import os

op_file_path = '/content/drive/MyDrive/Colab Notebooks/P_HINTS6/op_data/hints6_public_filtered_v2.xlsx'
sheet_name = 'df_cleaned'

if os.path.exists(op_file_path):
    with pd.ExcelWriter(op_file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)
else:
    df.to_excel(op_file_path, sheet_name=sheet_name, index=False)
